<a href="https://colab.research.google.com/github/aradhana148/Medical-NER-RE/blob/master/radgraph_ct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q \
  torch>=2.1.0 \
  transformers>=4.39.0 \
  appdirs \
  jsonpickle \
  filelock \
  h5py \
  nltk \
  dotmap \
  pytest


In [2]:
!pip install radgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.0/588.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for radgraph: filename=radgraph-0.1.18-py3-none-any.whl size=812635 sha256=b6bb610f38ff3eab376fed493ab9d39c4833b97b74ff1cdf809e0b526629bb55
  Stored in directory: /root/.cache/pip/wheels/fb/3c/fb/214f5d5cdab2a0f9f0904fd81d7fd1134404100b4444554df8
Successfully built radgraph


In [3]:
import json
from radgraph import get_radgraph_processed_annotations, RadGraph

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
import pandas as pd
import ast
import re


In [28]:
def clean_text(x):
    if not x:
        return ""
    if isinstance(x, list):
        return ", ".join(x)
    return str(x)


def annotation_to_sentence(annotation):
    obs = clean_text(annotation.get("observation"))
    loc = clean_text(annotation.get("located_at"))
    sug = clean_text(annotation.get("suggestive_of"))
    tag = annotation.get("tags", [""])[0]

    # Normalize tag
    tag = re.sub("_", " ", tag)

    modified_obs = obs # Initialize modified_obs

    parts = []

    # Handle ABSENT case first
    if tag == "definitely absent":
        if obs:
            # As per instruction: 'No {observation}' or 'No {observation} in the {location}'
            sentence = f"No {obs}"
            if loc:
                sentence += f" in the {loc}"
            sentence += "."
            return sentence.capitalize()
        elif loc: # If no observation but there's a location, assume "no abnormality"
            return f"No abnormality in the {loc}.".capitalize()
        else:
            return "" # Cannot form a coherent sentence for "absent" without observation or location

    # --- Handle PRESENT/DESCRIPTIVE tags ---
    # Handle other descriptive tags that directly modify the observation
    if tag.startswith("definitely ") and tag not in ["definitely absent", "definitely present"]:
        descriptive_tag = tag.replace("definitely ", "").strip()
        if modified_obs:
            modified_obs = f"{descriptive_tag} {modified_obs}"
        else:
            # If original obs was empty, add "findings" to the descriptive tag
            modified_obs = f"{descriptive_tag} findings"
    elif tag and tag not in ["definitely present", "definitely absent", "present", "absent", "uncertain", "likely", "possibly"]:
        if modified_obs:
            modified_obs = f"{tag} {modified_obs}"
        else:
            # If original obs was empty, add "findings" to the descriptive tag
            modified_obs = f"{tag} findings"

    # Combine suggestive_of with observation
    if modified_obs and sug:
        modified_obs = f"{modified_obs} suggestive of {sug}"
    elif not modified_obs and sug: # If modified_obs is empty and sug is present
        modified_obs = f"findings suggestive of {sug}"

    # PRESENT (or unknown) case
    if modified_obs:
        parts.append(modified_obs.capitalize())
        if loc:
            parts.append(f"in the {loc}") # Append location only if observation/suggestion is present

    sentence = " ".join(parts).strip()

    if sentence and not sentence.endswith("."):
        sentence += "."

    return sentence

In [38]:

df = pd.read_csv("/content/ct_data.csv")

clean = (
    df.iloc[10:20, 0]
      .astype(str)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
      .reset_index(drop=True)
)

concepts = df.iloc[10:20, 1].reset_index(drop=True)

eval_df = pd.DataFrame({"report": clean, "concepts": concepts})
model_type = "modern-radgraph-xl"
radgraph = RadGraph(model_type=model_type)
reports = eval_df["report"].astype(str).tolist()

pred_concepts = []

for report in reports:
    annotations = radgraph([report])  # one report batch
    processed = get_radgraph_processed_annotations(annotations)  # dict with "processed_annotations"

    sents = []
    for ann in processed["processed_annotations"]:
        s = annotation_to_sentence(ann)
        if s:
            sents.append(s)

    pred_concepts.append(list(dict.fromkeys(sents)))

reports = eval_df["report"].astype(str).tolist()

gt_concepts = [
    ast.literal_eval(x) if pd.notna(x) else []
    for x in eval_df["concepts"].tolist()
]


Using device: cpu


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [39]:
for i in range(len(pred_concepts)):
  print(i)
  print(reports[i])
  print(gt_concepts[i])
  print(pred_concepts[i])

0
The trachea and lumens of both main bronchi are open with no occlusive pathology detected. Patchy ground glass densities are observed in the inferior lingula of the left upper lobe. Prominent vascular structures are noted. Atelectatic changes are present in the middle lobe of the right lung. The heart contour and size are normal. No pericardial thickening or effusion is detected. Thoracic esophagus is normal in caliber with no significant wall thickening. No lymph nodes of pathological size or appearance are detected in the mediastinal and bilateral hilar regions. Several lymph nodes measuring up to 15x10 mm are present in both axillary regions. No lytic or destructive lesions are detected in the bone structures. The upper abdominal organs are partially included in the study and are evaluated as suboptimal.
['open trachea', 'open bronchi', 'patchy ground glass densities', 'prominent vascular structures', 'atelectatic changes', 'normal heart', 'no pericardial thickening', 'no pericard

In [40]:
def canonicalize_concept(s: str) -> list[str]:
    s = str(s).strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = s.rstrip(".")

    # Split concepts by comma for adjective propagation - now applied uniformly
    split_concepts = [c.strip() for c in s.split(',')]
    processed_concepts = []

    # Capture initial adjective for propagation, if available in the first part
    initial_adjective = ""
    if split_concepts:
        match_initial_adj = re.match(r"^(\w+)", split_concepts[0])
        if match_initial_adj:
            initial_adjective = match_initial_adj.group(1)

    for i, concept_str_raw in enumerate(split_concepts):
        concept_str = concept_str_raw

        # Adjective Propagation for subsequent parts
        if i > 0 and initial_adjective and \
           not re.match(r"^(no|normal|mild|moderate|severe|slight|minimal|increased|decreased|diffuse|bilateral|patchy|clear|enlarged|stable|unchanged|intact|present|absent)", concept_str.split(' ')[0]):
            concept_str = f"{initial_adjective} {concept_str}"

        # Canonicalization rules
        concept_str = concept_str.replace("top - normal", "normal")
        concept_str = concept_str.replace("within normal limits", "normal")
        concept_str = concept_str.replace("unremarkable", "normal")

        # Reorder "no/normal <observation> in the <location>" to "no/normal <location> <observation>"
        m = re.match(r"^(no|normal) (.+?) in the (.+)$", concept_str)
        if m:
            modifier = m.group(1)
            obs = m.group(2)
            loc = m.group(3)
            concept_str = f"{modifier} {loc} {obs}"
        else:
            # Refined reordering for general "X in the Y" patterns to "Y X" (e.g., "effusion in the pleural" -> "pleural effusion")
            # This now prioritizes location before observation for better alignment with GT
            m = re.match(r"^(.+?) in the (.+)$", concept_str)
            if m:
                obs, loc = m.group(1), m.group(2)
                concept_str = f"{loc} {obs}"

        concept_str = re.sub(r"\s+", " ", concept_str).strip()

        # Rule for "no lungs" -> "no"
        if concept_str == "no lungs":
            concept_str = "no"

        # Rule for removing 'lungs' if it appears alone after a descriptive term
        if concept_str.endswith(" lungs") and len(concept_str.split()) > 1:
            last_word_before_lungs = concept_str.split()[-2]
            if last_word_before_lungs in ["normal", "clear", "mild", "diffuse", "bilateral", "patchy", "both", "patent", "open", "few", "multiple", "several", "subcentimeter", "widespread", "all"]:
                concept_str = concept_str.rsplit(' ', 1)[0] # Remove " lungs"

        # Refined single-word leftover dropping
        # Expanded meaningful_single_words list
        meaningful_single_words = [
            "normal", "patent", "open", "clear", "present", "absent", "pneumothorax", "effusion", "consolidation",
            "atelectasis", "mass", "nodule", "lesion", "fibrosis", "emphysema", "infiltrates", "thickening",
            "dilatation", "abnormality", "spleen", "trachea", "bronchi", "heart", "aorta", "esophagus",
            "mediastinum", "axilla", "carina", "subcarinal", "adrenal", "liver", "bone", "pneumonia",
            "infection", "vascular", "structures", "cysts", "scarring", "fluid", "collection", "size",
            "contour", "caliber", "diameter", "heights", "margins", "wall", "tubes", "catheter", "implant",
            "opacities", "ground-glass", "density", "findings", "progression", "regression", "stable", "changes"
        ]
        meaningful_single_words = list(set(meaningful_single_words)) # Remove duplicates

        if len(concept_str.split()) == 1 and concept_str not in meaningful_single_words:
            concept_str = ""

        if concept_str: # Only add non-empty concepts
            processed_concepts.append(concept_str)

    return processed_concepts

gt_norm= []
for concept_list in gt_concepts:
    report_concepts = []
    for concept in concept_list:
        report_concepts.extend(canonicalize_concept(concept))
    gt_norm.append([x for x in report_concepts if x])

pred_norm= []
for concept_list in pred_concepts:
    report_concepts = []
    for concept_str_raw in concept_list:
        report_concepts.extend(canonicalize_concept(concept_str_raw))
    pred_norm.append([x for x in report_concepts if x])

In [41]:
for i in (range(len(pred_concepts))):
  print(reports[i])
  print(i," :")
  print(gt_norm[i],'\n',pred_norm[i],'\n')


The trachea and lumens of both main bronchi are open with no occlusive pathology detected. Patchy ground glass densities are observed in the inferior lingula of the left upper lobe. Prominent vascular structures are noted. Atelectatic changes are present in the middle lobe of the right lung. The heart contour and size are normal. No pericardial thickening or effusion is detected. Thoracic esophagus is normal in caliber with no significant wall thickening. No lymph nodes of pathological size or appearance are detected in the mediastinal and bilateral hilar regions. Several lymph nodes measuring up to 15x10 mm are present in both axillary regions. No lytic or destructive lesions are detected in the bone structures. The upper abdominal organs are partially included in the study and are evaluated as suboptimal.
0  :
['open trachea', 'open bronchi', 'patchy ground glass densities', 'prominent vascular structures', 'atelectatic changes', 'normal heart', 'no pericardial thickening', 'no peric

In [42]:
def similarity2(a: str, b: str) -> float:
    a_tokens = set(a.lower().split())
    b_tokens = set(b.lower().split())

    if not a_tokens and not b_tokens:
        return 1.0
    if not a_tokens or not b_tokens:
        return 0.0

    intersection = a_tokens & b_tokens
    union = a_tokens | b_tokens

    return (2*len(intersection)) / (len(a_tokens)+len(b_tokens))


In [43]:
def fuzzy_counts_one(gt_list, pred_list, threshold):
    gt = [g.strip().lower() for g in gt_list if g.strip()]
    pr = [p.strip().lower() for p in pred_list if p.strip()]

    used_gt = set()
    TP = 0

    for p in pr:
        best_j = None
        best_score = 0.0

        for j, g in enumerate(gt):
            if j in used_gt:
                continue

            score = similarity2(p, g)
            if score > best_score:
                best_score = score
                best_j = j

        if best_j is not None and best_score >= threshold:
            TP += 1
            used_gt.add(best_j)

    FP = len(pr) - TP
    FN = len(gt) - TP

    return TP, FP, FN


In [44]:

def fuzzy_prf(gt_norm, pred_norm, threshold):
    TP = FP = FN = 0

    for gt, pr in zip(gt_norm, pred_norm):
        t, f, n = fuzzy_counts_one(gt, pr, threshold)
        TP += t
        FP += f
        FN += n

    precision = TP / (TP + FP) if TP + FP else 0
    recall    = TP / (TP + FN) if TP + FN else 0
    f1        = (2 * precision * recall / (precision + recall)) if precision + recall else 0

    return precision, recall, f1, (TP, FP, FN)


In [45]:


for th in [0.7,0.75, 0.8, 0.85,0.9]:
    P, R, F1, counts = fuzzy_prf(gt_norm, pred_norm, threshold=th)
    print(th, P, R, F1)

0.7 0.4074074074074074 0.532258064516129 0.46153846153846145
0.75 0.3950617283950617 0.5161290322580645 0.44755244755244755
0.8 0.37037037037037035 0.4838709677419355 0.4195804195804196
0.85 0.2839506172839506 0.3709677419354839 0.32167832167832167
0.9 0.25308641975308643 0.33064516129032256 0.2867132867132867


In [46]:
import pandas as pd

# Create a DataFrame from pred_norm
pred_norm_df = pd.DataFrame({'concepts': pred_norm})

# Save the DataFrame to a CSV file
pred_norm_df.to_csv('pred_ct.csv', index=False)

print("CSV file 'pred_norm_concepts.csv' created successfully!")

CSV file 'pred_norm_concepts.csv' created successfully!
